In [3]:
from contextlib import contextmanager
import os
import datetime as dt
import numpy as np
import h5py
import time
import sys
os.chdir('../data_acquisition')
sys.path.append('../')
sys.path.append('../utils_meinhold/')
sys.path.append('../utils_zonca/')
from glob import glob

#sys.path.append('../../')
#sys.path.append('../../../')
sys.path.append('../VtoT/')

import Tkinter,tkFileDialog

import cofe_util as cu
import realtime_gp as rt
import datetime

from Tkinter import *
import ttk

from scipy import optimize
import pickle

In [4]:
def nametochan(name):
    #function to convert channel numbers to channel names

    #names of each channel
    chans = {
        'all': 'all',  'H1HiAC':'ch0',  'H1HiDC':'ch1',
        'H1LoAC':'ch2' ,  'H1LoDC':'ch3', 'H2HiAC':'ch4' ,
        'H2HiDC':'ch5' ,  'H2LoAC':'ch6',  'H2LoDC':'ch7',
        'H3HiAC':'ch8', 'H3HiDC':'ch9',  'H3LoAC':'ch10',
         'H3LoDC':'ch11', 'HornTop':'ch12', 'Amplifier': 'ch13',
        'Cooler':'ch14', 'Transition':'ch15'}

    chan = chans[name]

    return chan

In [5]:
%pylab

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


c:\users\labuser\anaconda3\envs\py27\lib\site-packages\IPython\core\magics\pylab.py:161: UserWarning: pylab import has clobbered these variables: ['Widget', 'Text', 'Button']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [6]:
def plot_some_chans2(datatype='demod',plottype='toi',filelist=None,components=['T','Q','U'],samprate=30,minfreq=.1, linearize=False):
    
    if filelist==None:
        root=Tkinter.Tk()
        if datatype=='demod':
            filelist = list(tkFileDialog.askopenfilenames(\
            initialdir='C://Greenland/greenpol/telescope_control/data_aquisition/demod_data/',parent=root,title='Choose a set of files'))
        if datatype=='raw':
            filelist = list(tkFileDialog.askopenfilenames(\
            initialdir='C://Greenland/greenpol/telescope_control/data_aquisition/data/',parent=root,title='Choose a set of files'))            
        root.destroy()
    filelist.sort()
    main = Tk()
    main.title("Choose channels to plot")
    main.geometry("+50+150")
    frame = ttk.Frame(main, padding=(3, 3, 12, 12))
    frame.grid(column=0, row=0, sticky=(N, S, E, W))

    #set up the listbox to choose what chans to plot
    chan_labels = StringVar()
    chan_labels.set("H1HiAC H1HiDC H1LoAC H1LoDC H2HiAC H2HiDC H2LoAC H2LoDC\
                 H3HiAC H3HiDC H3LoAC H3LoDC HornTop Amplifier Cooler Transition")           
    tlist=[]
    if datatype=='demod':
        dlist=[]
        for f in filelist:
            h=np.float64(f[-11:-9])
            m=np.float64(f[-9:-7])
            s=np.float64(f[-7:-5])
            hf=h5py.File(f)
            t=h+m/60.+(s+(hf['demod_data']['rev']-hf['demod_data']['rev'][0])/1000.)/3600.
            tlist.append(t)  #just generate a time to plot relative to
            dlist.append(hf['demod_data'])
        d=concatenate(dlist)
        hf.close() 
        
    if datatype=='raw':
        h=np.float64(filelist[0][-11:-9])
        m=np.float64(filelist[0][-9:-7])
        s=np.float64(filelist[0][-7:-5])
        d=rt.demod.read_raw(filelist)
        t=h+m/60.+(s+(d['rev']-d['rev'][0])/1000.)/3600.
        tlist.append(t)
    ut=concatenate(tlist)
    #figure()
    #plot(ut)
    lstbox = Listbox(frame,listvariable=chan_labels, selectmode=MULTIPLE, width=20, height=16)
    lstbox.grid(column=0, row=0, columnspan=2)
    
    lstbox2 = Listbox(frame,listvariable=chan_labels, selectmode=MULTIPLE, width=20, height=16)
    lstbox2.grid(column=2, row=0, columnspan=2)
    #start the plot

    fname=filelist[0]
    yyyy=os.path.dirname(filelist[0])[-8:-4]
    mm=os.path.dirname(filelist[0])[-4:-2]
    dd=os.path.dirname(filelist[0])[-2:]

    def select():
        reslist = list()
        selection = lstbox.curselection()
        figure()
        if datatype=='demod':
            title('%s TOI from %s %s %s' %(components,mm,dd,yyyy))
        if datatype=='raw':
            title('TOI from %s %s %s' %(mm,dd,yyyy))

        for i in selection:
            entry = lstbox.get(i)
            reslist.append(entry)
        for val in reslist:
            chan=nametochan(val)
            if datatype=='demod':
                for component in components:
                    if linearize == True:
                        d[chan][component] = rt.linearize_Vexp(d[chan][component], int(val[1]))
                    plot(ut,d[chan][component],label=val+' %s' %component)
            if datatype=='raw':
                if linearize == True:
                    d[chan] = rt.linearize_Vexp(d[chan], int(val[1]))
                plot(d[chan].flatten(),label=val)
            xlabel('Samples')
            ylabel('Output [v]')
            legend()

    def selectfft():
        reslist2 = list()
        selection2 = lstbox2.curselection()
        figure()
        if datatype=='demod':
            title('%s ASD from %s %s %s' %(components,mm,dd,yyyy))
        if datatype=='raw':
            title('ASD from %s %s %s' %(mm,dd,yyyy))
                

        for i in selection2:
            entry2 = lstbox2.get(i)
            reslist2.append(entry2)
        for val in reslist2:
            chan=nametochan(val)
            if datatype=='demod':
                for component in components:
                    if linearize == True:
                        d[chan][component] = rt.linearize_Vexp(d[chan][component], int(val[1]))
                    freq,psd=cu.nps(d[chan][component],samprate,minfreq=minfreq)
                    plot(freq,sqrt(psd),label=val+' %s' %component)
            if datatype=='raw':
                if linearize == True:
                    d[chan] = rt.linearize_Vexp(d[chan], int(val[1]))
                freq,psd=cu.nps(d[chan].flatten(),samprate*256,minfreq=minfreq)
                plot(freq,sqrt(psd),label=val)
            xlabel('Frequency [Hz]')
            ylabel(r'ASD [$\frac{V}{\sqrt{Hz}}$]')
            legend()
                
    btn = ttk.Button(frame, text="Select for TOI", command=select)
    btn.grid(column=1, row=1)
    
    btn2 = ttk.Button(frame, text="Select for ASD", command=selectfft)
    btn2.grid(column=3, row=1)
    
    main.mainloop()

In [7]:
def read_some_data(datadict,datalabel,datatype='demod', filelist=None):

    if filelist==None:
        root=Tkinter.Tk()
        if datatype=='demod':
            filelist = list(tkFileDialog.askopenfilenames(\
            initialdir='D://software_git_repos/greenpol/telescope_control/data_aquisition/demod_data/',parent=root,title='Choose a set of files'))
        if datatype=='raw':
            filelist = list(tkFileDialog.askopenfilenames(\
            initialdir='D://software_git_repos/greenpol/telescope_control/data_aquisition/data/',parent=root,title='Choose a set of files'))            
        root.destroy()
    filelist.sort()
    
    if datatype=='demod':
        dlist=[]
        for f in filelist:
            hf=h5py.File(f)
            dlist.append(hf['demod_data'])
        d=concatenate(dlist)
        hf.close() 
        
    if datatype=='raw':
        d=rt.demod.read_raw(filelist)
    datadict[datalabel]=d
    return

In [8]:
def plot_nl_dict(toi_dict,chanlist=['H1HiDC'],componentlist=['T'],datalist=None):
    #function to give a choice of preloaded datasets from dictionary data_dict
    #this plots chosen channels vs H1LoDC (should be attenuated more linear detector)
    figure()
    if datalist==None:
        datalist=toi_dict.keys()
    if len(componentlist)==1:  # if componentlist is a single value, assume same thing for all chans
        c=componentlist[0]
        componentlist=[c for n in chanlist]
    #set up the color sequence for the plot
    nlines=len(datalist)*len(chanlist)
    colormap = cm.gist_ncar
    gca().set_color_cycle([colormap(i) for i in np.linspace(0, 0.9, nlines)])
    titletxt=[]
    for dname in datalist:
        samprate=1000./(toi_dict[dname]['rev'][1]-toi_dict[dname]['rev'][0])
        print(dname,samprate)
        titletxt.append(dname)
        for channame,component in zip(chanlist,componentlist):
            print(channame,component)
            try:
                toi=toi_dict[dname][nametochan(channame)][component] #if it's demod data so a dict
                toi_x=toi_dict[dname][nametochan('H2LoDC')][component] #highly attenuated channel
                plot(toi_x,toi,label=dname+ '  '+channame+' '+component)
            except:
                toi=toi_dict[dname][nametochan(channame)].flatten()#if it's raw data so not a dict
                toi_x=toi_dict[dname][nametochan('H2LoDC')].flatten()# highly attenuated channel
                plot(toi_x,toi,label=dname+ '  '+channame+' '+component)
            
    title(titletxt)
    legend()
    xlabel('H1LoDC [V]')
    ylabel(r'Signal [V]')

In [9]:
def plot_toi_dict(toi_dict,chanlist=['H1HiAC'],componentlist=['T'],datalist=None, kperv=[1]):
    #function to give a choice of preloaded datasets from dictionary data_dict
    figure()
    if datalist==None:
        datalist=toi_dict.keys()
    if len(componentlist)==1:  # if componentlist is a single value, assume same thing for all chans
        c=componentlist[0]
        componentlist=[c for n in chanlist]
    #set up the color sequence for the plot
    nlines=len(datalist)*len(chanlist)
    colormap = cm.gist_ncar
    gca().set_color_cycle([colormap(i) for i in np.linspace(0, 0.9, nlines)])
    titletxt=[]
    count = 0
    for dname in datalist:
        samprate=1000./(toi_dict[dname]['rev'][1]-toi_dict[dname]['rev'][0])
        print(dname,samprate)
        titletxt.append(dname)
        for channame,component in zip(chanlist,componentlist):
            print(channame,component)
            try:
                toi=toi_dict[dname][nametochan(channame)][component] #if it's demod data so a dict
                t=toi_dict[dname]['azi'][0]+arange(len(toi))/samprate
                if len(kperv)==1 and kperv[0]==1.0:
                    plot(t,toi*kperv[count],label=dname+ '  '+channame+' '+component,lw=2)
                else:
                    plot(t,toi*kperv[count],label=dname+ '  '+channame+' '+component+' gain= '+str(kperv[count]),lw=2)
                if (count+1) < len(kperv):
                    count += 1
            except:
                toi=toi_dict[dname][nametochan(channame)].flatten()#if it's raw data so not a dict
                t=toi_dict[dname]['azi'][0]+arange(len(toi))/(256.*samprate)
                if len(kperv)==1 and kperv[0]==1.0:
                    plot(t,toi*kperv[count],label=dname+ '  '+channame+' '+component,lw=2)
                else:
                    plot(t,toi*kperv[count],label=dname+ '  '+channame+' '+component+' gain= '+str(kperv[count]),lw=2) 
                if (count+1) < len(kperv):
                    count += 1
                
    title(titletxt)
    legend()
    xlabel('Time [S]')
    if len(kperv) == 1 and kperv[0] == 1:
        ylabel('Output [v]')
    else:
        ylabel('Output [T]')

In [10]:
def asd_from_dict(data_dict,minfreq=1.):
    #run nps on all the channels for each of the loaded data sets, return asd dictionary
    asd_dict={}
    chans=['ch%s' %str(i) for i in range(16)]
    for datset in data_dict.keys():
        asd_dict[datset]={}
        datatype='demod'
        print(datset)
        d=data_dict[datset]
        if len(d.dtype)==18:
            datatype='raw'
        samprate=1000./float(d['rev'][10]-d['rev'][9])
        for chan in chans:
            if datatype=='raw':
                freq,psd=cu.nps(d[chan].flatten(),samprate*256,minfreq=minfreq)
                asd_dict[datset]['freq']=freq
                asd_dict[datset][chan]=sqrt(psd)
            if datatype=='demod':
                asd_dict[datset][chan]={}
                for component in ['T','Q','U']:
                    freq,psd=cu.nps(d[chan][component],samprate,minfreq=minfreq)
                    asd_dict[datset][chan][component]=sqrt(psd)
                    asd_dict[datset]['freq']=freq
                    
    return asd_dict            

In [11]:
def plot_asd_dict(asd_dict,chanlist=['H1HiAC'],componentlist=['T'],datalist=None,kperv=[1]):
    #function to give a choice of preloaded datasets from dictionary data_dict
    figure()
    if datalist==None:
        datalist=asd_dict.keys()
    if len(componentlist)==1:  # if componentlist is a single value, assume same thing for all chans
        c=componentlist[0]
        componentlist=[c for n in datalist]
    #set up the color sequence for the plot
    nlines=len(datalist)*len(chanlist)
    colormap = cm.gist_ncar
    gca().set_color_cycle([colormap(i) for i in np.linspace(0, 0.9, nlines)])
    
    count = 0
    for dname in datalist:
        for channame,component in zip(chanlist,componentlist):
            try:
                asd=asd_dict[dname][nametochan(channame)][component] #if it's demod data so a dict
            except:
                asd=asd_dict[dname][nametochan(channame)]           #if it's raw data so not a dict
            if len(kperv) == 1 and kperv[0] == 1.0:
                plot(asd_dict[dname]['freq'],asd*kperv[count],label=dname+ '  '+channame+' '+component,lw=2)
            else:
                plot(asd_dict[dname]['freq'],asd*kperv[count],label=dname+ '  '+channame+' '+component+' gain= '+str(kperv[count]),lw=2)
            if (count+1) < len(kperv):
                count += 1
    legend()
    xlabel('Frequency [Hz]')
    if len(kperv) == 1 and kperv[0] == 1:
        ylabel(r'ASD [$\frac{V}{\sqrt{Hz}}$]')
    else:
        ylabel(r'ASD [$\frac{T}{\sqrt{Hz}}$]')

In [10]:
plot_some_chans2(datatype='raw',components = 'T', samprate=30,minfreq=0.1,linearize=False)

IndexError: list index out of range

In [60]:
ddict={}

In [62]:
read_some_data(ddict,'NRM_feetbolted',datatype='raw')

1001
16


1000
16


1000
16


In [63]:
ddict.keys()

['NRM_feetbolted', 'NRM']

In [64]:
asd_dict = asd_from_dict(ddict, minfreq=0.1)

NRM_feetbolted
NRM


In [68]:
figure()
'''
asd=asd_dict['deton_everythingoff'][rt.nametochan('H1HiAC')] 
plot(asd_dict['deton_everythingoff']['freq'],asd,label='everything turned off but detector',lw=2)

v120 = 1.8e-5
l120 = np.zeros(len(asd))
l120[:] = v120
plot(asd_dict['deton_everythingoff']['freq'], l120, label = r'everything off, asd(120 Hz) = %f $\frac{V}{\sqrt{Hz}}$' % v120, lw=1)

asd=asd_dict['deton_azelon_boltin'][rt.nametochan('H1HiAC')] 
plot(asd_dict['deton_azelon_boltin']['freq'],asd,label='az/el motor on, bolt in frame',lw=2)

v120 = 1.8e-5
l120 = np.zeros(len(asd))
l120[:] = v120
plot(asd_dict['deton_azelon_boltin']['freq'], l120, label = r'az/el motor on, bolt in frame, asd(120 Hz) = %f $\frac{V}{\sqrt{Hz}}$' % v120, lw=1)

asd=asd_dict['deton_azelon_boltout'][rt.nametochan('H1HiAC')] 
plot(asd_dict['deton_azelon_boltout']['freq'],asd,label='elevation = 60 deg',lw=2)

v120 = 3.3e-4
l120 = np.zeros(len(asd))
l120[:] = v120
plot(asd_dict['deton_azelon_boltout']['freq'], l120, label = r'elevation = 60 deg, asd(120 Hz) = %f $\frac{V}{\sqrt{Hz}}$' % v120, lw=1)

asd=asd_dict['deton_azelon_boltout_el37'][rt.nametochan('H1HiAC')] 
plot(asd_dict['deton_azelon_boltout_el37']['freq'],asd,label='elevation = 37 deg',lw=2)

v120 = 1.4e-4
l120 = np.zeros(len(asd))
l120[:] = v120
plot(asd_dict['deton_azelon_boltout_el37']['freq'], l120, label = r'elevation = 37 deg, asd(120 Hz) = %f $\frac{V}{\sqrt{Hz}}$' % v120, lw=1)
'''

#asd=asd_dict['NRM_feetbolted'][rt.nametochan('H1HiAC')]*1000*4.4/np.sqrt(2)
#plot(asd_dict['NRM_feetbolted']['freq'],asd,label='feet bolted',lw=2)

#v120 = 1000*1.3e-2*4.4/np.sqrt(2)
#l120 = np.zeros(len(asd))
#l120[:] = v120
#plot(asd_dict['NRM_feetbolted']['freq'], l120, label = r'feet bolted, asd(120 Hz) = %.2f $\mu K \sqrt{sec}$' % v120, lw=1)



asd=asd_dict['NRM'][rt.nametochan('H1HiAC')]*1000*4.4/np.sqrt(2)
plot(asd_dict['NRM']['freq'],asd,label='feet unbolted',lw=2)

v120 = 1000*5.5e-4*4.4/np.sqrt(2)
l120 = np.zeros(len(asd))
l120[:] = v120
plot(asd_dict['NRM']['freq'], l120, label = 'feet unbolted, asd(120 Hz) = %.2f $\mu K \sqrt{sec}$' % v120, lw=1)


legend()
title('Normal Running Mode (az on, el off, encoder)')
xlabel('Frequency [Hz]')
ylabel(r'ASD [$\mu K \sqrt{sec}$]')  

Text(0,0.5,'ASD [$\\mu K \\sqrt{sec}$]')

In [65]:

figure()

asd=asd_dict['elonazonel60_amb'][rt.nametochan('H1HiAC')] 
plot(asd_dict['elonazonel60_amb']['freq'],asd,label='motor on, el = 60 deg: H1HiAC T',lw=2)

l120 = np.zeros(len(asd))
l120[:] = 4e-2
plot(asd_dict['elonazonel60_amb']['freq'], l120, label = r'motor on, el = $60^{\degree}$, asd(120 Hz) = %f $\frac{V}{\sqrt{Hz}}$' % 4e-2, lw=3)

asd=asd_dict['eloffazoffel60_amb'][rt.nametochan('H1HiAC')] 
plot(asd_dict['eloffazoffel60_amb']['freq'],asd,label='motor off, el = $60^{\degree}$: H1HiAC T',lw=2)

l120[:] =3.4e-2
plot(asd_dict['eloffazoffel60_amb']['freq'], l120, label = r'motor off, el = $60^{\degree}$, asd(120 Hz) = %f $\frac{V}{\sqrt{Hz}}$' % 3.4e-2, lw=3)

asd=asd_dict['elonazonel37_amb'][rt.nametochan('H1HiAC')] 
plot(asd_dict['elonazonel37_amb']['freq'],asd,label='motor on, el = $37^{\degree}$: H1HiAC T',lw=2)

l120[:] = 3.2e-3
plot(asd_dict['elonazonel37_amb']['freq'], l120, label = r'motor on w/ bolt, el = $37^{\degree}$, asd(120 Hz) = %f $\frac{V}{\sqrt{Hz}}$' % 3.2e-3, lw=3)

asd=asd_dict['elonazonel37_nobolt_amb'][rt.nametochan('H1HiAC')] 
plot(asd_dict['elonazonel37_nobolt_amb']['freq'],asd,label='motor on, el = $37^{\degree}$: H1HiAC T',lw=2)

l120[:] = 5.5e-2
plot(asd_dict['elonazonel37_nobolt_amb']['freq'], l120, label = r'motor on, el = $37^{\degree}$, asd(120 Hz) = %f $\frac{V}{\sqrt{Hz}}$' % 5.5e-2, lw=3)


asd=asd_dict['eloffazoffel37_amb'][rt.nametochan('H1HiAC')] 
plot(asd_dict['eloffazoffel37_amb']['freq'],asd,label='motor off, el = $37^{\degree}$: H1HiAC T',lw=2)

l120[:] = 2.5e-3
plot(asd_dict['eloffazoffel37_amb']['freq'], l120, label = r'motor off, el = $37^{\degree}$, asd(120 Hz) = %f $\frac{V}{\sqrt{Hz}}$' % 2.5e-3, lw=3)



legend()
title('Motor on/off + elevation comparisons looking at ambient load')
xlabel('Frequency [Hz]')
ylabel(r'ASD [$\frac{V}{\sqrt{Hz}}$]')     

In [118]:
plot_asd_dict(asd_dict,chanlist=['H1HiAC'],componentlist=['T'],datalist=['eloffazonel60','sky_no_filters'])

C:\Users\labuser\Anaconda\envs\py27\lib\site-packages\ipykernel\__main__.py:18: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [16]:
plot_toi_dict(ddict,chanlist=['H1HiDC'],componentlist=['T'],datalist=['H3in', 'H3out', 'H3in_cal', 'H3out_cal'])

('H3in', 31.25)
('H1HiDC', 'T')
('H3out', 29.41176470588235)
('H1HiDC', 'T')
('H3in_cal', 31.25)
('H1HiDC', 'T')
('H3out_cal', 31.25)
('H1HiDC', 'T')


c:\users\labuser\anaconda3\envs\py27\lib\site-packages\ipykernel_launcher.py:12: MatplotlibDeprecationWarning: The set_color_cycle function was deprecated in version 1.5. Use `.set_prop_cycle` instead.
  if sys.path[0] == '':


In [29]:
from demod import datparsing

In [118]:
beta_h1=(ddict['nocalch1'][nametochan('H1HiAC')]['Q'].mean()-ddict['pcalch1'][nametochan('H1HiAC')]['Q'].mean())/ddict['nocalch1'][nametochan('H1HiAC')]['Q'].mean()
beta_h2=(ddict['nocalch2'][nametochan('H2HiAC')]['Q'].mean()-ddict['pcalch2'][nametochan('H2HiAC')]['Q'].mean())/ddict['nocalch2'][nametochan('H2HiAC')]['Q'].mean()
beta_h3=(ddict['nocalch3'][nametochan('H3HiAC')]['Q'].mean()-ddict['pcalch3'][nametochan('H3HiAC')]['Q'].mean())/ddict['nocalch3'][nametochan('H3HiAC')]['Q'].mean()
print('Horn 1 transmission fraction',beta_h1)
print('Horn 2 transmission fraction',beta_h2)
print('Horn 3 transmission fraction',beta_h3)

('Horn 1 transmission fraction', 0.12419560576729863)
('Horn 2 transmission fraction', 0.12628101445342926)
('Horn 3 transmission fraction', 0.11703026443075649)


In [39]:
from collections import defaultdict

In [16]:
horns = [1,2,3,1]
Vamb = [-1.81, -0.824, -0.77, -1.796]
Vpar = [-0.901, -0.393, -0.367, -0.915]
VLN = [-0.722, -0.32, -0.298, -0.74]
beta = [0.124, 0.126, 0.117, 0.124]

asd = [5.1e6, 9.99e6, 9.7e6, 5.1e6]

h1gamb = []
h1gpar = []
h1pdiff = []

Tamb = 269.
TLN = 75.
Tsky = 10.
Tsys = 10.

print 'LN cal applied to relatively clear sky data'
print '___________________________________________'
print ' '

for h in range(0, 4):
    Tpar = beta[h]*Tamb + (1-beta[h])*TLN
    
    Vamb_lin = rt.linearize_Vexp(Vamb[h], horns[h])
    Vpar_lin = rt.linearize_Vexp(Vpar[h], horns[h])
    VLN_lin = rt.linearize_Vexp(VLN[h], horns[h])
    
    pdiffamb = 100*(Vamb_lin - Vamb[h])/Vamb[h]
    pdiffpar = 100*(Vpar_lin - Vpar[h])/Vpar[h]
    pdiffLN = 100*(VLN_lin - VLN[h])/VLN[h]
    
    gamb = (Tamb - TLN)/(Vamb_lin - VLN_lin)
    gpar = (Tpar - TLN)/(Vpar_lin - VLN_lin)
    
    nl = -1e6*(asd[h]*1e-9*gamb/30.)/np.sqrt(2)
    
    y = Vpar_lin/VLN_lin
    
    Tsys = (Tpar - y*TLN)/(y-1)
    
    enl = 1e6*(Tsys + Tamb)/np.sqrt(4e9)
    
    pdiffgain = 100*(gamb - gpar)/gamb
    
    if h == 0 or h == 3:
        h1gamb.append(gamb)
        h1gpar.append(gpar)
        h1pdiff.append(pdiffgain)
    
    print 'horn %d percent difference between compressed and linearized ambient voltage: %f' % (horns[h], pdiffamb)
    print 'horn %d percent difference between compressed and linearized partial cal voltage: %f' % (horns[h], pdiffpar)
    print 'horn %d percent difference between compressed and linearized LN voltage: %f' % (horns[h], pdiffLN)
    print 'horn %d ambient to LN gain: %f K/V' % (horns[h], gamb)
    print 'horn %d partial cal to LN gain: %f K/V' % (horns[h], gpar)
    print 'horn %d percent difference between partial cal and ambient cal gains: %f' % (horns[h], pdiffgain)
    #print 'horn %d y factor: ' % horns[h], y
    #print 'horn %d Tsys: ' % horns[h], Tsys
    print 'horn %d noise level: %f muK sqrt(sec)' % (horns[h], nl)
    print 'horn %d noise level expected with 4 GHz bandpass: %f muK sqrt(sec)' % (horns[h], enl)
    print 'horn %d noise level at pre-amp: %f nV/sqrt(Hz)' % (horns[h], asd[h]/100./30.)
    print ' '
    print '--------------------------------------------------------------------------------------------------'
    print ' '

print 'horn 1 percent difference between 1st and last ambient-LN gain: %f' % (100*abs((h1gamb[0] - h1gamb[1])/h1gamb[0]))
print 'horn 1 percent difference between 1st and last partial-LN gain: %f' % (100*abs((h1gpar[0] - h1gpar[1])/h1gpar[0]))
print 'horn 1 percent difference between 1st and last amb-partial gain percent difference: %f' % (100*abs((h1pdiff[0] - h1pdiff[1])/h1pdiff[0]))

LN cal applied to relatively clear sky data
___________________________________________
 
horn 1 percent difference between compressed and linearized ambient voltage: 30.286326
horn 1 percent difference between compressed and linearized partial cal voltage: 14.448888
horn 1 percent difference between compressed and linearized LN voltage: 11.270586
horn 1 ambient to LN gain: -124.774178 K/V
horn 1 partial cal to LN gain: -105.596374 K/V
horn 1 percent difference between partial cal and ambient cal gains: 15.370011
horn 1 noise level: 14998.873501 muK sqrt(sec)
horn 1 noise level expected with 4 GHz bandpass: 4408.742242 muK sqrt(sec)
horn 1 noise level at pre-amp: 1700.000000 nV/sqrt(Hz)
 
--------------------------------------------------------------------------------------------------
 
horn 2 percent difference between compressed and linearized ambient voltage: 4.724564
horn 2 percent difference between compressed and linearized partial cal voltage: -0.197620
horn 2 percent differenc

In [ ]:
horns = [1,2,3]
asd = [3.42e6, 1.66e6, 4.2e6]

pgains = [-395.035239897, -867.289055377, -412.044076708]

for h in horns:
    Vamb1 = ddict['amb1'][nametochan('H%dHiDC' % h)].mean()
    Vamb1 = rt.linearize_Vexp(Vamb1, h)

    Vamb3 = ddict['amb3'][nametochan('H%dHiDC' % h)].mean()
    Vamb3 = rt.linearize_Vexp(Vamb3, h)

    gainratio = Vamb1/Vamb3
    print 'gain ratio: ', gainratio
    gain = pgains[h-1]*gainratio
    
    nl = -1e3*(asd[h-1]*1e-9*gain/30.)/np.sqrt(2)
    
    enl = 1e3*(10 + 300)/np.sqrt(4e9)

    print 'horn %d gain: ' % h, gain
    print 'horn %d noise level: %f mK sqrt(sec)' % (h, nl)
    print 'horn %d noise level expected with 4 GHz bandpass and Tsys 10K: %f mK sqrt(sec)' % (h, enl)
    print ' '

In [13]:
horns = [1,2,3]
asd = [485221, 482591, 1.46893e6]
Th = 271
Tc = 10

for h in horns:
    vsky = ddict['sky'][nametochan('H%dHiDC' % h)].mean()
    #vpar = ddict['H%d_par' % h][nametochan('H%dHiDC' % h)]['T'].mean()
    vamb = ddict['amb'][nametochan('H%dHiDC' % h)].mean()
    
    vskylin = rt.linearize_Vexp(vsky, h)
    #vpar = rt.linearize_Vexp(vpar, h)
    vamblin = rt.linearize_Vexp(vamb, h)
    
    y = vamblin/vskylin
    
    g = (Th - Tc)/(vamblin - vskylin)
    
    Tsys = (Th - y*Tc)/(y-1)
    
    psd = -1e6*(asd[h-1]*1e-9*g/30.)/np.sqrt(2)
    
    print 'horn %d gain' % h, g
    print 'horn %d Tsys' % h, Tsys
    print 'horn %d noise level' %h, psd, ' muK sqrt(sec)'
    print ' '
    

horn 1 gain -119.603594258
horn 1 Tsys 5.15229528783
horn 1 noise level 1367.87863713  muK sqrt(sec)
 
horn 2 gain -352.548181589
horn 2 Tsys 3.73831756339
horn 2 noise level 4010.15763644  muK sqrt(sec)
 
horn 3 gain -381.940502622
horn 3 Tsys 10.9147489866
horn 3 noise level 13223.9306576  muK sqrt(sec)
 


In [14]:
horns = [1,2,3,1]
Vamb = [-1.717, -0.873, -0.6675, -1.73]
Vpar = [-0.8355, -0.404, -0.33, -0.844]
VLN = [-0.6545, -0.3205, -0.264, -0.66]
beta = [0.124, 0.126, 0.117, 0.124]

asd = [485221, 482591, 1.46893e6, 485221]

h1gamb = []
h1gpar = []
h1pdiff = []

Tamb = 271.
TLN = 75.
Tsky = 10.
Tsys = 10.

print 'LN cal applied to relatively clear sky data'
print '___________________________________________'
print ' '

for h in range(0, 4):
    Tpar = beta[h]*Tamb + (1-beta[h])*TLN
    
    Vamb_lin = rt.linearize_Vexp(Vamb[h], horns[h])
    Vpar_lin = rt.linearize_Vexp(Vpar[h], horns[h])
    VLN_lin = rt.linearize_Vexp(VLN[h], horns[h])
    
    pdiffamb = 100*(Vamb_lin - Vamb[h])/Vamb[h]
    pdiffpar = 100*(Vpar_lin - Vpar[h])/Vpar[h]
    pdiffLN = 100*(VLN_lin - VLN[h])/VLN[h]
    
    gamb = (Tamb - TLN)/(Vamb_lin - VLN_lin)
    gpar = (Tpar - TLN)/(Vpar_lin - VLN_lin)
    
    nl = -1e6*(asd[h]*1e-9*gpar/30.)/np.sqrt(2)
    
    #y = Vpar_lin/VLN_lin
    
    #Tsys = (Tpar - y*TLN)/(y-1)
    
    enl = 1e6*(Tsys + Tsky)/np.sqrt(4e9)
    
    pdiffgain = 100*(gamb - gpar)/gamb
    
    if h == 0 or h == 3:
        h1gamb.append(gamb)
        h1gpar.append(gpar)
        h1pdiff.append(pdiffgain)
    
    print 'horn %d percent difference between compressed and linearized ambient voltage: %f' % (horns[h], pdiffamb)
    print 'horn %d percent difference between compressed and linearized partial cal voltage: %f' % (horns[h], pdiffpar)
    print 'horn %d percent difference between compressed and linearized LN voltage: %f' % (horns[h], pdiffLN)
    print 'horn %d ambient to LN gain: %f K/V' % (horns[h], gamb)
    print 'horn %d partial cal to LN gain: %f K/V' % (horns[h], gpar)
    print 'horn %d percent difference between partial cal and ambient cal gains: %f' % (horns[h], pdiffgain)
    #print 'horn %d y factor: ' % horns[h], y
    #print 'horn %d Tsys: ' % horns[h], Tsys
    print 'horn %d noise level: %f muK sqrt(sec)' % (horns[h], nl)
    print 'horn %d noise level expected with 4 GHz bandpass: %f muK sqrt(sec)' % (horns[h], enl)
    print 'horn %d noise level at pre-amp: %f nV/sqrt(Hz)' % (horns[h], asd[h]/100./30.)
    print ' '
    print '--------------------------------------------------------------------------------------------------'
    print ' '

print 'horn 1 percent difference between 1st and last ambient-LN gain: %f' % (100*abs((h1gamb[0] - h1gamb[1])/h1gamb[0]))
print 'horn 1 percent difference between 1st and last partial-LN gain: %f' % (100*abs((h1gpar[0] - h1gpar[1])/h1gpar[0]))
print 'horn 1 percent difference between 1st and last amb-partial gain percent difference: %f' % (100*abs((h1pdiff[0] - h1pdiff[1])/h1pdiff[0]))

LN cal applied to relatively clear sky data
___________________________________________
 
horn 1 percent difference between compressed and linearized ambient voltage: 28.709687
horn 1 percent difference between compressed and linearized partial cal voltage: 13.288885
horn 1 percent difference between compressed and linearized LN voltage: 10.061357
horn 1 ambient to LN gain: -131.579500 K/V
horn 1 partial cal to LN gain: -107.455640 K/V
horn 1 percent difference between partial cal and ambient cal gains: 18.334056
horn 1 noise level: 1228.945298 muK sqrt(sec)
horn 1 noise level expected with 4 GHz bandpass: 316.227766 muK sqrt(sec)
horn 1 noise level at pre-amp: 161.740333 nV/sqrt(Hz)
 
--------------------------------------------------------------------------------------------------
 
horn 2 percent difference between compressed and linearized ambient voltage: 5.267623
horn 2 percent difference between compressed and linearized partial cal voltage: -0.061516
horn 2 percent difference b

In [51]:
horns = [1,]
Th = 300
Tc = 20

for h in horns:
    vLN = ddict['sky'][nametochan('H%dHiDC' % h)].mean()
    #vpar = ddict['H%d_par' % h][nametochan('H%dHiDC' % h)]['T'].mean()
    vamb = ddict['amb'][nametochan('H%dHiDC' % h)].mean()
    
    vLN = rt.linearize_Vexp(vLN, h)
    #vpar = rt.linearize_Vexp(vpar, h)
    vamblin = rt.linearize_Vexp(vamb, h)
    
    #y = vamb/vLN
    
    g = (Th - Tc)/(vamb - vLN)
    
    #Tsys = (Th - y*Tc)/(y-1)
    
    #print 'Horn %d LN: ' % h, vLN, 'V' 
    #print 'Horn %d partial cal: ' % h, vpar, 'V' 
    print 'Horn %d sky: ' % h, vamb, 'V' 
    print 'Horn %d amb: ' % h, vamblin, 'V' 
    print 'Horn %d gain: ' % h, g, 'K/V' 
    #print 'Horn %d y factor: ' % h, y 
    #print 'Horn %d Tsys: ' % h, Tsys, 'K' 
    print ''
    
    

Horn 1 sky:  -0.398295236255 V
Horn 1 sky:  -0.419327295856 V
Horn 1 gain:  124.875543416 K/V



In [38]:
import demod
dd = demod.readraw(fld)

AttributeError: 'module' object has no attribute 'readraw'

In [43]:
print rt.linearize_Vexp(-2.9,3)

-3.91153631274


In [126]:
Vhigh=rt.linearize_Vexp(-1.724,1)
print(Vhigh)
Vlow=rt.linearize_Vexp(-.516,1)
print(Vlow)
print(Vhigh/Vlow)

-2.21791124688
-0.550969987297
4.02546653723


In [18]:
horns = [1,2,3]
for h in horns:
    Vamb = rt.linearize_Vexp(ddict['amb'][nametochan('H%dHiDC' % h)].mean(),h)
    #Vpar = rt.linearize_Vexp(ddict['par'][nametochan('H1HiDC')]['T'].mean(),1,2)
    #Vln = rt.linearize_Vexp(ddict['ln'][nametochan('H1HiDC')]['T'].mean(),1,2)
    Vsky = rt.linearize_Vexp(ddict['sky'][nametochan('H%dHiDC' % h)].mean(),h)

    Th = 261
    Tc = 20

    gain = (Th-Tc)/(Vamb - Vsky)

    #y = Vamb/Vln

    #Tpar = (Vpar - Vln)*gain

    #Tsys = (Th - y*Tc)/(y-1)
    
    print 'horn gain: ', gain
    #print gain
    #print Tpar
    #print Tsys

horn gain:  -112.098340927
horn gain:  -336.134714762
horn gain:  -684.525729075


In [97]:
var = 'Q'
figure()
for key in rdict.keys():
    if (key[-1]=='1'):
        channame='H1HiDC'
        title(channame)
        toi=rdict[key][nametochan(channame)].flatten()#
        stoi=mean(rdict[key][nametochan(channame)],axis=0)
        plot(stoi,label=key) 
        legend()
"""
    if (key[-1]=='2'):
        channame='H2HiAC'
        title(channame)
        toi=rdict[key][nametochan(channame)].flatten()#
        stoi=mean(rdict[key][nametochan(channame)],axis=0)
        plot(stoi,label=key) 
        legend()

    if (key[-1]=='3'):
        channame='H3HiAC'
        title(channame)
        toi=rdict[key][nametochan(channame)].flatten()#
        stoi=mean(rdict[key][nametochan(channame)],axis=0)
        plot(stoi,label=key) 
        legend()
"""

"\n    if (key[-1]=='2'):\n        channame='H2HiAC'\n        title(channame)\n        toi=rdict[key][nametochan(channame)].flatten()#\n        stoi=mean(rdict[key][nametochan(channame)],axis=0)\n        plot(stoi,label=key) \n        legend()\n\n    if (key[-1]=='3'):\n        channame='H3HiAC'\n        title(channame)\n        toi=rdict[key][nametochan(channame)].flatten()#\n        stoi=mean(rdict[key][nametochan(channame)],axis=0)\n        plot(stoi,label=key) \n        legend()\n"

In [13]:
plot_toi_dict(rdict,chanlist=['H1HiDC', 'H1LoDC'],componentlist=['T'],datalist=['nl1_3_23'])

('nl1_3_23', 31.25)
('H1HiDC', 'T')
('H1LoDC', 'T')


C:\Users\labuser\Anaconda\envs\py27\lib\site-packages\ipykernel\__main__.py:21: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [12]:
plot_toi_dict(rdict,chanlist=['H1HiDC'],componentlist=['T'])

('pcalch1', 29.411764705882351)
('H1HiDC', 'T')
('nocalch1', 29.411764705882351)
('H1HiDC', 'T')


C:\Users\labuser\Anaconda\envs\py27\lib\site-packages\ipykernel\__main__.py:21: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [13]:
rdict.keys()

['nl1_3_22', 'nl2_3_22', 'nl3_3_23']

In [23]:
d1=rdict['nl1_3_22'][nametochan('H1HiDC')].flatten()
r1=rdict['nl1_3_22'][nametochan('H2LoDC')].flatten()
d2=rdict['nl2_3_22'][nametochan('H2HiDC')].flatten()
r2=rdict['nl2_3_22'][nametochan('H2LoDC')].flatten()
d3=rdict['nl3_3_23'][nametochan('H3HiDC')].flatten()
r3=rdict['nl3_3_23'][nametochan('H2LoDC')].flatten()

In [25]:
d1=rdict['nl1_3_22'][nametochan('H1LoDC')].flatten()
r1=rdict['nl1_3_22'][nametochan('H2LoDC')].flatten()
d2=rdict['nl2_3_22'][nametochan('H1LoDC')].flatten()
r2=rdict['nl2_3_22'][nametochan('H2LoDC')].flatten()
d3=rdict['nl3_3_23'][nametochan('H1LoDC')].flatten()
r3=rdict['nl3_3_23'][nametochan('H2LoDC')].flatten()

In [26]:
figure()
plot(r2,d2)
title('H2')

figure()
plot(r1,d1)
title('H1')

figure()
plot(r3,d3)
title('H3')

In [16]:
#take section of data with increasing power
d31=d3[505516:921971]
r31=r3[505516:921971]

d21 = d2[496965:1047710]
r21 = r2[496965:1047710]

d11 = d1[259609:604672]
r11 = r1[259609:604672]

#take section of data with decreasing power
d32 = d3[1017160:1520870]
r32 = r3[1017160:1520870]

d22 = d2[1154790:1766730]
r22 = r2[1154790:1766730]

d12 = d1[703828:1025090]
r12 = r1[703828:1025090]

In [22]:
l = len(d3)
figure()
plot(np.arange(l), d3)
plot(np.arange(l)[505516:921971], d31)
plot(np.arange(l)[1017160:1520870], d32)
title('H3')

l = len(d2)
#figure()
plot(np.arange(l), d2)
plot(np.arange(l)[496965:1047710], d21)
plot(np.arange(l)[1154790:1766730], d22)
title('H2')

l = len(d1)
#figure()
plot(np.arange(l), d1)
plot(np.arange(l)[259609:604672], d11)
plot(np.arange(l)[703828:1025090], d12)
title('H1')

In [17]:
isort=argsort(d1)
ds1=d1[isort]
rs1=r1[isort]
isort=argsort(d2)
ds2=d2[isort]
rs2=r2[isort]
isort=argsort(d3)
ds3=d3[isort]
rs3=r3[isort]

In [18]:
isort=argsort(d31)
ds31=d31[isort]
rs31=r31[isort]

isort=argsort(d32)
ds32=d32[isort]
rs32=r32[isort]

isort=argsort(d21)
ds21=d21[isort]
rs21=r21[isort]

isort=argsort(d22)
ds22=d22[isort]
rs22=r22[isort]

isort=argsort(d11)
ds11=d11[isort]
rs11=r11[isort]

isort=argsort(d12)
ds12=d12[isort]
rs12=r12[isort]

In [19]:
def compfunc(x,p):
    """
    define a non-linear function to fit compression
    parameters p(4): (offset,slope,exponential coefficient, exponent)
    """
    y=p[0]+p[1]*x+p[2]*exp(p[3]*x)
    return y

def compfuncresiduals(p,y,x):
    err=y-compfunc(x,p)
    return err

In [20]:
def fit_compression(x,y,pstart=None):
    """
    function to fit compression function
    """
    xmax=x[y==np.max(y)][0]
    if pstart==None:
        pstart=[np.max(y),xmax,1,1]
        if np.max(y)==y[0]:
            pstart=[y[0],x[0]-1,0,1]
        if np.max(y)==y[-1]:
            pstart=[y[-1],x[-1]+1,1,0]
    pbest=optimize.leastsq(compfuncresiduals,pstart,args=(y,x))
    params={}
    params['offset']=pbest[0][0]
    params['slope']=pbest[0][1]
    params['exp_coeff']=np.abs(pbest[0][2])
    params['exponent']=np.abs(pbest[0][3])
    params['fit']=compfunc(x,pbest[0])
    return pbest[0]

In [21]:
expars1=fit_compression(rs1,ds1)
expars2=fit_compression(rs2,ds2)
expars3=fit_compression(rs3,ds3)

In [22]:
expars31=fit_compression(rs31,ds31)
expars32=fit_compression(rs32,ds32)

expars21=fit_compression(rs21,ds21)
expars22=fit_compression(rs22,ds22)

expars11=fit_compression(rs11,ds11)
expars12=fit_compression(rs12,ds12)

In [23]:
dsf1=np.array([ds1[i] for i in range(len(ds1)) if (ds1[i]>-.5) and (ds1[i]<.04)])
rsf1=np.array([rs1[i] for i in range(len(ds1)) if (ds1[i]>-.5) and (ds1[i]<.04)])
dsf2=np.array([ds2[i] for i in range(len(ds2)) if (ds2[i]>-.5) and (ds2[i]<.04)])
rsf2=np.array([rs2[i] for i in range(len(ds2)) if (ds2[i]>-.5) and (ds2[i]<.04)])
dsf3=np.array([ds3[i] for i in range(len(ds3)) if (ds3[i]>-.5) and (ds3[i]<.04)])
rsf3=np.array([rs3[i] for i in range(len(ds3)) if (ds3[i]>-.5) and (ds3[i]<.04)])

In [24]:
dsf31=np.array([ds31[i] for i in range(len(ds31)) if (ds31[i]>-.5) and (ds31[i]<.04)])
rsf31=np.array([rs31[i] for i in range(len(ds31)) if (ds31[i]>-.5) and (ds31[i]<.04)])
dsf32=np.array([ds32[i] for i in range(len(ds32)) if (ds32[i]>-.5) and (ds32[i]<.04)])
rsf32=np.array([rs32[i] for i in range(len(ds32)) if (ds32[i]>-.5) and (ds32[i]<.04)])

dsf21=np.array([ds21[i] for i in range(len(ds21)) if (ds21[i]>-.5) and (ds21[i]<.04)])
rsf21=np.array([rs21[i] for i in range(len(ds21)) if (ds21[i]>-.5) and (ds21[i]<.04)])
dsf22=np.array([ds22[i] for i in range(len(ds22)) if (ds22[i]>-.5) and (ds22[i]<.04)])
rsf22=np.array([rs22[i] for i in range(len(ds22)) if (ds22[i]>-.5) and (ds22[i]<.04)])

dsf11=np.array([ds11[i] for i in range(len(ds11)) if (ds11[i]>-.5) and (ds11[i]<.04)])
rsf11=np.array([rs11[i] for i in range(len(ds11)) if (ds11[i]>-.5) and (ds11[i]<.04)])
dsf12=np.array([ds12[i] for i in range(len(ds12)) if (ds12[i]>-.5) and (ds12[i]<.04)])
rsf12=np.array([rs12[i] for i in range(len(ds12)) if (ds12[i]>-.5) and (ds12[i]<.04)])

In [25]:
pars1=cu.linfit(dsf1,rsf1)
pars2=cu.linfit(dsf2,rsf2)
pars3=cu.linfit(dsf3,rsf3)

In [26]:
pars31=cu.linfit(dsf31,rsf31)
pars32=cu.linfit(dsf32,rsf32)

pars21=cu.linfit(dsf21,rsf21)
pars22=cu.linfit(dsf22,rsf22)

pars11=cu.linfit(dsf11,rsf11)
pars12=cu.linfit(dsf12,rsf12)

In [27]:
print(expars1)
print(pars1)
print(expars2)
print(pars2)
print(expars3)
print(pars3)

[ -0.92490326   9.50300791   0.74546308  11.24411128]
[ 0.00944183  0.05265621]
[ -1.63474924  15.64453405   1.39352799  12.20774018]
[ 0.00710002  0.03012872]
[-0.45968616  5.06249856  0.38960185  8.36641406]
[ 0.007721    0.12555349]


In [55]:
pars={'expars1':expars12,'expars2':expars22,'expars3':expars32, 'linpars1':pars12,
     'linpars2':pars22,'linpars3':pars32}

fpath='D:/software_git_repos/greenpol/telescope_control/configurations'
#fpath='D:/software_git_repos/greenpol/telescope_control/configurations'
os.chdir(fpath)

folder='calibrations'

#get label for save file
fname='fit_params_decreasing'

os.chdir(fpath+'/'+folder)

if os.path.isfile(fname+'.txt')==True:
    print "LABEL EXISTS. Please change your label!"

else:
    with open(fname+'.txt', 'w') as handle:
        pickle.dump(pars,handle)
os.chdir('D:/software_git_repos/greenpol/telescope_control/data_aquisition')

In [28]:
print(expars31)
print(pars31)

print(expars32)
print(pars32)

[ -0.50001041   5.15714164   0.41825985  16.52559029]
[ 0.00915773  0.09925225]
[ -0.10473639   5.96772233   0.26495333  28.78710669]
[-0.00763016  0.13233211]


In [29]:
dsmod1=(rs1-pars1[0])/pars1[1]
dsmod2=(rs2-pars2[0])/pars2[1]
dsmod3=(rs3-pars3[0])/pars3[1]

In [30]:
dsmod31=(rs31-pars31[0])/pars31[1]
dsmod32=(rs32-pars32[0])/pars32[1]

dsmod21=(rs21-pars21[0])/pars21[1]
dsmod22=(rs22-pars22[0])/pars22[1]

dsmod11=(rs11-pars11[0])/pars11[1]
dsmod12=(rs12-pars12[0])/pars12[1]

In [31]:
#Vlin1=linearize_Vexp(ds1,lkp1)
#Vlin2=linearize_Vexp(ds2,lkp2)
#Vlin3=linearize_Vexp(ds3,lkp3)
Vlin1=rt.linearize_Vexp(ds1,1, params=2)
Vlin2=rt.linearize_Vexp(ds2,2, params=2)
Vlin3=rt.linearize_Vexp(ds3,3, params=2)

In [42]:
print rt.linearize_Vexp(-0.7, 1, '_decreasing')

-0.747820401174


In [33]:
Vlin31=rt.linearize_Vexp(ds31,3, params='_increasing')
Vlin32=rt.linearize_Vexp(ds32,3, params='_decreasing')

Vlin21=rt.linearize_Vexp(ds21,2, params='_increasing')
Vlin22=rt.linearize_Vexp(ds22,2, params='_decreasing')

Vlin11=rt.linearize_Vexp(ds11,1, params='_increasing')
Vlin12=rt.linearize_Vexp(ds12,1, params='_decreasing')

In [34]:
V1exp = -1.8
print rt.linearize_Vexp(V1exp,1, params='_increasing')
print rt.linearize_Vexp(V1exp,1, params='_decreasing')
print rt.linearize_Vexp(V1exp,1, params=2)

print ' '
V2exp = -1.99
print rt.linearize_Vexp(V2exp,2, params='_increasing')
print rt.linearize_Vexp(V2exp,2, params='_decreasing')
print rt.linearize_Vexp(V2exp,2, params=2)

print ' '
V3exp = -2.16
print rt.linearize_Vexp(V3exp,3, params='_increasing')
print rt.linearize_Vexp(V3exp,3, params='_decreasing')
print rt.linearize_Vexp(V3exp,3, params=2)

-1.98323185837
-2.34069096335
-2.34211726217
 
-2.39884925609
-2.36772963332
-2.34851639966
 
-3.33930941567
-2.54487514619
-2.77211802125


In [46]:
'''
figure()
title('Horn 3 Hi')

plot(rs3,ds3,label='data')
plot(rs31,ds31,label='increasing')
plot(rs32,ds32,label='decreasing')

expmod31=compfunc(rs31,expars31)
expmod32=compfunc(rs32,expars32)

plot(rs31,expmod31,label='exponential Fit (increasing)')
plot(rs32,expmod32,label='exponential Fit (decreasing)')
plot(rs3,expmod3,label='exponential Fit')


#plot(rsf31,dsf31,label='data for linear fit 1')
#plot(rsf32,dsf32,label='data for linear fit 2')

plot(rs31,Vlin31,label='linearized data (increasing)')
plot(rs32,Vlin32,label='linearized data (decreasing)')
plot(rs3,Vlin3,label='linearized data total')

plot(rs31,dsmod31,label='Linear Fit (increasing)')
plot(rs32,dsmod32,label='Linear Fit (decreasing)')
plot(rs3,dsmod3,label='Linear Fit total')

ylabel('Detector voltage (x100)')
xlabel('Ref voltage (H1LoDC)')
legend()

figure()
title('Horn 2 Hi')

plot(rs2,ds2,label='data')
plot(rs21,ds21,label='increasing')
plot(rs22,ds22,label='decreasing')

expmod21=compfunc(rs21,expars21)
expmod22=compfunc(rs22,expars22)

plot(rs21,expmod21,label='exponential Fit (increasing)')
plot(rs22,expmod22,label='exponential Fit (decreasing)')
plot(rs2,expmod2,label='exponential Fit')


#plot(rsf21,dsf21,label='data for linear fit increasing')
#plot(rsf22,dsf22,label='data for linear fit decreasing')

plot(rs21,Vlin21,label='linearized data (increasing)')
plot(rs22,Vlin22,label='linearized data (decreasing)')
plot(rs2,Vlin2,label='linearized data total')

plot(rs21,dsmod21,label='Linear Fit (increasing)')
plot(rs22,dsmod22,label='Linear Fit (decreasing)')
plot(rs2,dsmod2,label='Linear Fit total')

ylabel('Detector voltage (x100)')
xlabel('Ref voltage (H1LoDC)')
legend()
'''
figure()
title('Horn 1 Hi')

#plot(rs1,ds1,label='data')
#plot(rs11,ds11,label='data increasing')
plot(rs12,ds12,label='data decreasing')

#expmod11=compfunc(rs11,expars11)
expmod12=compfunc(rs12,expars12)

#plot(rs11,expmod11,label='exponential Fit (increasing)')
plot(rs12,expmod12,label='exponential Fit (decreasing)')
#plot(rs1,expmod1,label='exponential Fit')


#plot(rsf11,dsf11,label='data for linear fit increasing')
#plot(rsf12,dsf12,label='data for linear fit decreasing')

#plot(rs11,Vlin11,label='linearized data (increasing)')
plot(rs12,Vlin12,label='linearized data (decreasing)')
#plot(rs1,Vlin1,label='linearized data total')

#plot(rs11,dsmod11,label='Linear Fit (increasing)')
plot(rs12,dsmod12,label='Linear Fit (decreasing)')
#plot(rs1,dsmod1,label='Linear Fit total')

ylabel('Detector voltage (x100)')
xlabel('Ref voltage (H1LoDC)')
legend()

In [36]:

figure()
title('Horn 1 Hi')
plot(rs1,ds1,label='data')
expmod1=compfunc(rs1,expars1)
plot(rsf1,dsf1,label='data for linear fit')
plot(rs1,Vlin1,label='linearized data')
plot(rs1,expmod1,label='exponential Fit')
plot(rs1,dsmod1,label='Linear Fit')

ylabel('Detector voltage (x100)')
xlabel('Ref voltage (H1LoDC)')
legend()

figure()
title('Horn 2 Hi')
plot(rs2,ds2,label='data')
expmod2=compfunc(rs2,expars2)
plot(rsf2,dsf2,label='data for linear fit')
plot(rs2,Vlin2,label='linearized data')
plot(rs2,expmod2,label='exponential Fit')
plot(rs2,dsmod2,label='Linear Fit')
ylabel('Detector voltage (x100)')
xlabel('Ref voltage (H1LoDC)')
legend()

figure()
title('Horn 3 Hi')
plot(rs3,ds3,label='data')
expmod3=compfunc(rs3,expars3)
plot(rsf3,dsf3,label='data for linear fit')
plot(rs3,Vlin3,label='linearized data')
plot(rs3,expmod3,label='exponential Fit')
plot(rs3,dsmod3,label='Linear Fit')
ylabel('Detector voltage (x100)')
xlabel('Ref voltage (H1LoDC)')
legend()

In [37]:
figure()
plot(lkp[0],lkp[1])

In [71]:
figure()
plot(dsf1)

In [17]:
plot_some_chans2(datatype='raw',minfreq=.1)

1001
16


1000
16


1000
16


1000
16


1000
16


1000
16


1000
16


In [77]:
figure()
plot(r,d,'.')

In [63]:
figure()
plot(adict['nl_2a_src_3_21']['ch3'].flatten(),adict['nl_2a_src_3_21']['ch1'].flatten())

In [69]:
plot_toi_dict(rdict,chanlist=['H1HiDC','H1LoDC','H2LoDC'],componentlist=['T'],datalist=['nl_2b_src_3_21'])

('nl_2b_src_3_21', 31.25)
('H1HiDC', 'T')
('H1LoDC', 'T')
('H2LoDC', 'T')


C:\Users\labuser\Anaconda\envs\py27\lib\site-packages\ipykernel\__main__.py:21: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
